# TMT (Total Miscoordination Time) Analysis - Scenario 1 Only

This notebook analyzes only `scenario_1` (the base scenario). It loads the JSON data, filters to scenario_1, computes metrics, generates high-quality PNG plots, and saves CSV/JSON/TXT outputs under `results/`. All content is in English and HTML generation is disabled.


In [40]:
# %% Imports and Configuration
import json
import re
from pathlib import Path
from datetime import datetime
from collections import defaultdict

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

pio.templates.default = "plotly_white"
pio.renderers.default = "notebook_connected"

class ProjectConfig:
    """Portable configuration for sharing; paths auto-detected."""
    def __init__(self, data_file_name: str = "automation_results.json"):
        current_dir = Path.cwd()
        if current_dir.name == "notebooks":
            project_root = current_dir.parent
        elif current_dir.name == "analysis":
            project_root = current_dir.parent
        else:
            project_root = current_dir
        if not (project_root / "data" / "raw").exists():
            if current_dir.name == "notebooks" and current_dir.parent.name == "analysis":
                project_root = current_dir.parent.parent
            elif current_dir.name == "analysis":
                project_root = current_dir.parent
        self.PROJECT_ROOT = project_root
        self.DATA_DIR = project_root / "data"
        self.RAW_DATA_DIR = self.DATA_DIR / "raw"
        self.RESULTS_DIR = project_root / "results"
        self.TABLES_DIR = self.RESULTS_DIR / "tables"
        self.PLOTS_DIR = self.RESULTS_DIR / "plots" / "tmt_analysis"
        self.INPUT_FILE = self.RAW_DATA_DIR / data_file_name
        self.CTI = 0.2
        self.CRITICAL_TMT_THRESHOLD = -20.0
        self.MIN_COORDINATION_THRESHOLD = 80.0
        self.FIG_WIDTH = 1200
        self.FIG_HEIGHT = 600
        self.TEXT_ON_BAR_FONT_SIZE = 10
        self.PLOT_TEMPLATE = "plotly_white"
        self.COLORS = {
            'coordinated': '#2E8B57',
            'uncoordinated': '#DC143C',
            'critical': '#FF4500',
            'warning': '#FFD700',
            'good': '#32CD32',
            'primary': '#4169E1',
            'secondary': '#9370DB'
        }
        self.PLOTS_DIR.mkdir(parents=True, exist_ok=True)
        self.TABLES_DIR.mkdir(parents=True, exist_ok=True)

config = ProjectConfig()
print(f"📁 Project root: {config.PROJECT_ROOT}")
print(f"📊 Data file: {config.INPUT_FILE} (exists={config.INPUT_FILE.exists()})")


📁 Project root: /Users/gustavo/Documents/Projects/TESIS_UNAL/AutoDOC-MG
📊 Data file: /Users/gustavo/Documents/Projects/TESIS_UNAL/AutoDOC-MG/data/raw/automation_results.json (exists=True)


In [41]:
# %% Save filtered scenario_1 pairs for GA notebook
processed_dir = config.PROJECT_ROOT / "data" / "processed"
processed_dir.mkdir(parents=True, exist_ok=True)
scenario_pairs_path = processed_dir / "automation_results_scenario_1.json"
with open(scenario_pairs_path, 'w', encoding='utf-8') as f:
    json.dump(relay_pairs, f, indent=2)
print(f"✅ scenario_1 pairs saved for GA: {scenario_pairs_path}")


✅ scenario_1 pairs saved for GA: /Users/gustavo/Documents/Projects/TESIS_UNAL/AutoDOC-MG/data/processed/automation_results_scenario_1.json


In [42]:
# %% Load and Filter Data to scenario_1 Only

def validate_file_path(file_path: Path) -> None:
    if not file_path.exists() or not file_path.is_file():
        raise FileNotFoundError(f"File not found: {file_path}")

def load_relay_data(file_path: Path):
    validate_file_path(file_path)
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    if not isinstance(data, list):
        raise TypeError("Input JSON must be a list of relay pair entries")
    return data

def filter_to_scenario_1(relay_pairs: list):
    scenario = 'scenario_1'
    filtered = [p for p in relay_pairs if isinstance(p, dict) and p.get('scenario_id') == scenario]
    return scenario, filtered

print("🔄 Loading data...")
relay_pairs_all = load_relay_data(config.INPUT_FILE)
print(f"✅ Loaded {len(relay_pairs_all):,} entries from JSON")

scenario_id, relay_pairs = filter_to_scenario_1(relay_pairs_all)
print(f"✅ Filtered to {scenario_id}: {len(relay_pairs):,} entries")


🔄 Loading data...
✅ Loaded 6,800 entries from JSON
✅ Filtered to scenario_1: 100 entries


In [43]:
# %% Compute Metrics and DataFrame for scenario_1

def analyze_scenario(relay_pairs: list, cti: float):
    result = {
        'tmt': 0.0,
        'coordinated': 0,
        'uncoordinated': 0,
        'total_valid': 0,
        'time_differences': [],
        'miscoordination_times': []
    }

    for pair in relay_pairs:
        main, backup = pair.get('main_relay'), pair.get('backup_relay')
        if not isinstance(main, dict) or not isinstance(backup, dict):
            continue
        main_t, backup_t = main.get('Time_out'), backup.get('Time_out')
        if not isinstance(main_t, (int, float)) or not isinstance(backup_t, (int, float)):
            continue
        if main_t < 0 or backup_t < 0:
            continue
        delta_t = backup_t - main_t - cti
        mt = (delta_t - abs(delta_t)) / 2
        result['time_differences'].append(delta_t)
        if delta_t < 0:
            result['miscoordination_times'].append(abs(delta_t))
        result['tmt'] += mt
        result['total_valid'] += 1
        if delta_t >= 0:
            result['coordinated'] += 1
        else:
            result['uncoordinated'] += 1

    total = result['total_valid']
    pct_coord = (result['coordinated'] / total * 100) if total > 0 else 0.0
    avg_dt = float(np.mean(result['time_differences'])) if result['time_differences'] else 0.0
    std_dt = float(np.std(result['time_differences'])) if result['time_differences'] else 0.0
    max_miscoord = max(result['miscoordination_times']) if result['miscoordination_times'] else 0.0
    avg_miscoord = float(np.mean(result['miscoordination_times'])) if result['miscoordination_times'] else 0.0

    quality = 'GOOD' if pct_coord >= config.MIN_COORDINATION_THRESHOLD else 'REGULAR' if pct_coord >= 60 else 'CRITICAL'

    df = pd.DataFrame([{
        'Scenario': 'scenario_1',
        'TMT': result['tmt'],
        'Coordinated Pairs': result['coordinated'],
        'Uncoordinated Pairs': result['uncoordinated'],
        'Total Valid Pairs': total,
        'Coordination (%)': pct_coord,
        'Average Time Difference': avg_dt,
        'Std Time Difference': std_dt,
        'Max Miscoordination': max_miscoord,
        'Average Miscoordination': avg_miscoord,
        'Quality': quality
    }])

    metrics = {
        'total_pairs': total,
        'coordinated_pairs': result['coordinated'],
        'uncoordinated_pairs': result['uncoordinated'],
        'tmt': result['tmt'],
        'avg_coordination': pct_coord,
        'coordination_std': df['Coordination (%)'].std(),
        'tmt_std': df['TMT'].std(),
        'coordination_median': df['Coordination (%)'].median(),
        'tmt_median': df['TMT'].median(),
        'quality': quality
    }
    return df, metrics

df_s1, metrics_s1 = analyze_scenario(relay_pairs, config.CTI)
print("✅ Metrics computed for scenario_1")
print(df_s1)


✅ Metrics computed for scenario_1
     Scenario      TMT  Coordinated Pairs  Uncoordinated Pairs  \
0  scenario_1 -15.8543                 12                   88   

   Total Valid Pairs  Coordination (%)  Average Time Difference  \
0                100              12.0                -0.158408   

   Std Time Difference  Max Miscoordination  Average Miscoordination   Quality  
0             0.091403               0.3842                 0.180163  CRITICAL  


In [44]:
# %% Visualizations (PNG only)

def plot_coordination(df: pd.DataFrame):
    fig = px.bar(
        df,
        x='Scenario', y='Coordination (%)', color='Quality',
        color_discrete_map={'GOOD': config.COLORS['good'], 'REGULAR': config.COLORS['warning'], 'CRITICAL': config.COLORS['critical']},
        text='Coordination (%)',
        title='Coordination Quality - scenario_1',
        labels={'Scenario': 'Operational Scenario', 'Coordination (%)': 'Coordination (%)'}
    )
    fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside', textfont_size=config.TEXT_ON_BAR_FONT_SIZE)
    fig.update_layout(template=config.PLOT_TEMPLATE, width=config.FIG_WIDTH, height=config.FIG_HEIGHT)
    return fig


def plot_tmt(df: pd.DataFrame):
    df = df.copy()
    df['Severity'] = df['TMT'].apply(lambda x: 'CRITICAL' if x <= config.CRITICAL_TMT_THRESHOLD else 'HIGH' if x <= -10 else 'MODERATE' if x <= -5 else 'LOW' if x < 0 else 'NONE')
    fig = px.bar(
        df,
        x='Scenario', y='TMT', color='Severity',
        color_discrete_map={'CRITICAL': config.COLORS['critical'], 'HIGH': config.COLORS['uncoordinated'], 'MODERATE': config.COLORS['warning'], 'LOW': '#FFA500', 'NONE': config.COLORS['good']},
        text='TMT',
        title='TMT Severity - scenario_1',
        labels={'Scenario': 'Operational Scenario', 'TMT': 'Accumulated TMT (s)'}
    )
    fig.update_traces(texttemplate='%{text:.3f}', textposition='outside', textfont_size=config.TEXT_ON_BAR_FONT_SIZE)
    fig.update_layout(template=config.PLOT_TEMPLATE, width=config.FIG_WIDTH, height=config.FIG_HEIGHT)
    return fig

fig_coord = plot_coordination(df_s1)
fig_tmt = plot_tmt(df_s1)
print("✅ Figures created")


✅ Figures created


In [45]:
# %% Save Outputs (CSV, JSON metrics, TXT report, PNG plots)

def save_png(fig, name: str):
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    notebook_name = "tmt_analysis_scenario_1"
    png_path = config.PLOTS_DIR / f"{notebook_name}_{name}_{ts}.png"
    try:
        fig.write_image(png_path, scale=2, width=config.FIG_WIDTH, height=config.FIG_HEIGHT)
        print(f"✅ PNG saved: {png_path.name}")
    except Exception as e:
        print(f"⚠️  Could not save PNG ({name}): {e}")
        try:
            import kaleido
            fig.write_image(png_path, scale=2, width=config.FIG_WIDTH, height=config.FIG_HEIGHT)
            print(f"✅ PNG saved (with kaleido): {png_path.name}")
        except Exception as e2:
            print(f"❌ Error saving PNG: {e2}")

# Save plots
save_png(fig_coord, 'coordination_quality')
save_png(fig_tmt, 'tmt_severity')

# Save table
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
notebook_name = "tmt_analysis_scenario_1"
csv_path = config.TABLES_DIR / f"{notebook_name}_results_{ts}.csv"
df_s1.to_csv(csv_path, index=False, encoding='utf-8')
print(f"✅ CSV saved: {csv_path.name}")

# Save metrics JSON
metrics_path = config.TABLES_DIR / f"{notebook_name}_metrics_{ts}.json"
with open(metrics_path, 'w', encoding='utf-8') as f:
    json.dump(metrics_s1, f, indent=2)
print(f"✅ Metrics JSON saved: {metrics_path.name}")

# Save detailed TXT report
report_path = config.RESULTS_DIR / "reports"
report_path.mkdir(parents=True, exist_ok=True)
text_report = report_path / f"{notebook_name}_detailed_report_{ts}.txt"
with open(text_report, 'w', encoding='utf-8') as f:
    f.write("="*80 + "\n")
    f.write("DETAILED REPORT - scenario_1\n")
    f.write("="*80 + "\n\n")
    f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Project: {config.PROJECT_ROOT.name}\n")
    f.write(f"Source file: {config.INPUT_FILE.name}\n\n")
    f.write("METRICS\n")
    f.write("-"*20 + "\n")
    for k, v in metrics_s1.items():
        f.write(f"• {k}: {v}\n")
    f.write("\nDATA\n")
    f.write("-"*20 + "\n")
    f.write(df_s1.to_string(index=False))
print(f"✅ TXT report saved: {text_report.name}")


⚠️  Could not save PNG (coordination_quality): 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's instructions for your operating system,
or install it from your terminal by running:

    $ plotly_get_chrome


❌ Error saving PNG: 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's instructions for your operating system,
or install it from your terminal by running:

    $ plotly_get_chrome


⚠️  Could not save PNG (tmt_severity): 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's instructions for your operating system,
or install it from your terminal by running:

    $ plotly_get_chrome


❌ Error saving PNG: 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's instructions for your operating system,
or install it from your terminal by runnin

In [46]:
# %% Build per-pair DataFrame for scenario_1

def build_pair_level_dataframe(relay_pairs: list, cti: float) -> pd.DataFrame:
    rows = []
    for entry in relay_pairs:
        pair_id = entry.get('pair_id', None)
        main = entry.get('main_relay') or {}
        backup = entry.get('backup_relay') or {}
        main_t = main.get('Time_out', None)
        backup_t = backup.get('Time_out', None)
        if not isinstance(main_t, (int, float)) or not isinstance(backup_t, (int, float)):
            continue
        if main_t < 0 or backup_t < 0:
            continue
        delta_t = backup_t - main_t - cti
        coordinated = delta_t >= 0
        miscoord_mag = abs(delta_t) if delta_t < 0 else 0.0
        rows.append({
            'Pair ID': pair_id,
            'Main Time (s)': main_t,
            'Backup Time (s)': backup_t,
            'Δt = Backup - Main - CTI (s)': delta_t,
            'Coordinated': 'YES' if coordinated else 'NO',
            'Miscoordination (s)': miscoord_mag
        })
    df_pairs = pd.DataFrame(rows)
    if not df_pairs.empty:
        df_pairs = df_pairs.sort_values(by='Δt = Backup - Main - CTI (s)').reset_index(drop=True)
    return df_pairs

pair_df = build_pair_level_dataframe(relay_pairs, config.CTI)
print(f"✅ Pair-level dataset created: {len(pair_df):,} rows")
if not pair_df.empty:
    display(pair_df.head(10))
else:
    print("❌ No valid pairs found for scenario_1")


✅ Pair-level dataset created: 100 rows


,Pair ID,Main Time (s),Backup Time (s),Δt = Backup - Main - CTI (s),Coordinated,Miscoordination (s)
0,None,0.2699,0.0857,-0.3842,NO,0.3842
1,None,0.2806,0.0969,-0.3837,NO,0.3837
2,None,0.2676,0.0945,-0.3731,NO,0.3731
3,None,0.2732,0.1067,-0.3665,NO,0.3665
4,None,0.2420,0.0899,-0.3521,NO,0.3521
5,None,0.2664,0.1217,-0.3447,NO,0.3447
6,None,0.2566,0.1300,-0.3266,NO,0.3266
7,None,0.2540,0.1562,-0.2978,NO,0.2978
8,None,0.2112,0.1335,-0.2777,NO,0.2777
9,None,0.1676,0.0969,-0.2707,NO,0.2707


In [47]:
# %% Per-pair Visual Analysis
if not pair_df.empty:
    # 1) Histogram of Δt per pair
    fig_dt_hist = px.histogram(
        pair_df, x='Δt = Backup - Main - CTI (s)', nbins=30,
        title='Distribution of Δt (Backup - Main - CTI) - scenario_1',
        labels={'Δt = Backup - Main - CTI (s)': 'Δt (s)'}
    )
    fig_dt_hist.update_layout(template=config.PLOT_TEMPLATE, width=config.FIG_WIDTH, height=500)
    fig_dt_hist.show()

    # 2) Bar chart of miscoordination magnitude per pair (worst 25)
    worst = pair_df.sort_values('Miscoordination (s)', ascending=False).head(25)
    fig_miscoord = px.bar(
        worst, x='Pair ID', y='Miscoordination (s)',
        title='Top 25 Miscoordination Magnitudes - scenario_1',
        labels={'Pair ID': 'Pair ID', 'Miscoordination (s)': 'Miscoordination (s)'}
    )
    fig_miscoord.update_layout(template=config.PLOT_TEMPLATE, width=config.FIG_WIDTH, height=config.FIG_HEIGHT)
    fig_miscoord.show()

    # 3) Scatter of main vs backup times with coordination coloring
    fig_times = px.scatter(
        pair_df, x='Main Time (s)', y='Backup Time (s)', color='Coordinated',
        color_discrete_map={'YES': config.COLORS['good'], 'NO': config.COLORS['critical']},
        title='Main vs Backup Trip Times - scenario_1',
        labels={'Main Time (s)': 'Main Time (s)', 'Backup Time (s)': 'Backup Time (s)'}
    )
    fig_times.update_layout(template=config.PLOT_TEMPLATE, width=config.FIG_WIDTH, height=config.FIG_HEIGHT)
    fig_times.show()
else:
    print("❌ No pair-level data to visualize.")


In [48]:
# %% Display Figures Inline
print("📊 Showing figures inline...")
fig_coord.show()
fig_tmt.show()


📊 Showing figures inline...
